# کارٹ پول بیلنسنگ کے لیے RL کی تربیت

یہ نوٹ بک [AI for Beginners Curriculum](http://aka.ms/ai-beginners) کا حصہ ہے۔ یہ [PyTorch کے آفیشل ٹیوٹوریل](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) اور [کارٹ پول کے اس PyTorch امپلیمنٹیشن](https://github.com/yc930401/Actor-Critic-pytorch) سے متاثر ہو کر لکھی گئی ہے۔

اس مثال میں، ہم RL کا استعمال کرتے ہوئے ایک ماڈل کو تربیت دیں گے تاکہ وہ ایک کارٹ پر موجود پول کو بیلنس کر سکے، جو افقی سطح پر دائیں اور بائیں حرکت کر سکتا ہے۔ ہم [OpenAI Gym](https://www.gymlibrary.ml/) کے ماحول کا استعمال کریں گے تاکہ پول کی نقل و حرکت کو سیمولیٹ کیا جا سکے۔

> **نوٹ**: آپ اس سبق کے کوڈ کو مقامی طور پر (مثلاً Visual Studio Code سے) چلا سکتے ہیں، اس صورت میں سیمولیشن ایک نئی ونڈو میں کھلے گی۔ اگر آپ کوڈ کو آن لائن چلا رہے ہیں، تو آپ کو کوڈ میں کچھ تبدیلیاں کرنی پڑ سکتی ہیں، جیسا کہ [یہاں](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7) بیان کیا گیا ہے۔

ہم اس بات کو یقینی بنانے سے شروع کریں گے کہ Gym انسٹال ہے:


In [ ]:
import sys
!{sys.executable} -m pip install gym

اب آئیے CartPole ماحول بناتے ہیں اور دیکھتے ہیں کہ اس پر کیسے کام کیا جا سکتا ہے۔ ایک ماحول کی درج ذیل خصوصیات ہوتی ہیں:

* **ایکشن اسپیس** وہ ممکنہ اقدامات کا مجموعہ ہے جو ہم ہر قدم پر انجام دے سکتے ہیں۔
* **آبزرویشن اسپیس** وہ مشاہدات کا دائرہ ہے جو ہم کر سکتے ہیں۔


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

آئیے دیکھتے ہیں کہ سیمولیشن کیسے کام کرتی ہے۔ درج ذیل لوپ سیمولیشن کو چلاتا ہے، جب تک کہ `env.step` اختتامی پرچم `done` واپس نہ کرے۔ ہم `env.action_space.sample()` کا استعمال کرتے ہوئے بے ترتیب طور پر اعمال منتخب کریں گے، جس کا مطلب ہے کہ تجربہ شاید بہت جلد ناکام ہو جائے گا (CartPole ماحول اس وقت ختم ہو جاتا ہے جب CartPole کی رفتار، اس کی پوزیشن یا زاویہ مخصوص حدود سے باہر ہو)۔

> سیمولیشن ایک نئی ونڈو میں کھلے گی۔ آپ کوڈ کو کئی بار چلا سکتے ہیں اور دیکھ سکتے ہیں کہ یہ کیسے برتاؤ کرتا ہے۔


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

آپ دیکھ سکتے ہیں کہ مشاہدات میں چار نمبر شامل ہیں۔ یہ ہیں:  
- گاڑی کی پوزیشن  
- گاڑی کی رفتار  
- پول کا زاویہ  
- پول کی گردش کی شرح  

`rew` وہ انعام ہے جو ہمیں ہر قدم پر ملتا ہے۔ CartPole ماحول میں آپ کو ہر سیمولیشن قدم کے لیے 1 پوائنٹ کا انعام ملتا ہے، اور مقصد یہ ہے کہ کل انعام کو زیادہ سے زیادہ کیا جائے، یعنی CartPole کو گرنے کے بغیر زیادہ سے زیادہ وقت تک توازن برقرار رکھنے کے قابل بنایا جائے۔

ری انفورسمنٹ لرننگ کے دوران، ہمارا مقصد ایک **پالیسی** $\pi$ کو تربیت دینا ہے، جو ہر حالت $s$ کے لیے ہمیں بتائے گی کہ کون سا عمل $a$ لینا ہے، یعنی بنیادی طور پر $a = \pi(s)$۔

اگر آپ ایک احتمالی حل چاہتے ہیں، تو آپ پالیسی کو اس طرح سوچ سکتے ہیں کہ یہ ہر عمل کے لیے امکانات کا ایک سیٹ واپس کرے، یعنی $\pi(a|s)$ کا مطلب ہوگا کہ ہمیں حالت $s$ پر عمل $a$ لینا چاہیے اس کا امکان۔

## پالیسی گریڈینٹ طریقہ

سب سے سادہ RL الگورتھم، جسے **پالیسی گریڈینٹ** کہا جاتا ہے، میں ہم ایک نیورل نیٹ ورک کو تربیت دیں گے تاکہ اگلا عمل پیش گوئی کر سکے۔  


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

ہم نیٹ ورک کو تربیت دینے کے لیے کئی تجربات چلائیں گے، اور ہر تجربے کے بعد اپنے نیٹ ورک کو اپ ڈیٹ کریں گے۔ آئیے ایک فنکشن کی تعریف کرتے ہیں جو تجربہ چلائے گا اور نتائج واپس کرے گا (جسے **ٹریس** کہا جاتا ہے) - تمام حالتیں، اعمال (اور ان کی تجویز کردہ امکانات)، اور انعامات:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

آپ ایک غیر تربیت یافتہ نیٹ ورک کے ساتھ ایک قسط چلا سکتے ہیں اور مشاہدہ کر سکتے ہیں کہ کل انعام (یعنی قسط کی لمبائی) بہت کم ہے:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

پالیسی گریڈینٹ الگورتھم کے پیچیدہ پہلوؤں میں سے ایک **ڈسکاؤنٹڈ انعامات** کا استعمال ہے۔ خیال یہ ہے کہ ہم کھیل کے ہر مرحلے پر کل انعامات کے ویکٹر کا حساب لگاتے ہیں، اور اس عمل کے دوران ہم ابتدائی انعامات کو کسی عددی $gamma$ کے ذریعے ڈسکاؤنٹ کرتے ہیں۔ ہم نتیجے میں حاصل ہونے والے ویکٹر کو بھی معمول پر لاتے ہیں، کیونکہ ہم اسے اپنے تربیتی عمل پر اثر ڈالنے کے لیے وزن کے طور پر استعمال کریں گے:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

اب ہم اصل تربیت شروع کرتے ہیں! ہم 300 اقساط چلائیں گے، اور ہر قسط میں ہم درج ذیل کریں گے:

1. تجربہ چلائیں اور ٹریس جمع کریں۔
2. ان اعمال کے درمیان فرق (`gradients`) کا حساب لگائیں جو کیے گئے ہیں، اور پیش گوئی شدہ امکانات کے ذریعے۔ جتنا کم فرق ہوگا، اتنا ہی زیادہ یقین ہوگا کہ ہم نے صحیح عمل کیا ہے۔
3. ڈسکاؤنٹڈ انعامات کا حساب لگائیں اور `gradients` کو ڈسکاؤنٹڈ انعامات سے ضرب دیں - یہ یقینی بنائے گا کہ زیادہ انعام والے اقدامات کا حتمی نتیجے پر کم انعام والے اقدامات کے مقابلے میں زیادہ اثر ہوگا۔
4. ہمارے نیورل نیٹ ورک کے لیے متوقع ہدفی اعمال جزوی طور پر دوڑ کے دوران پیش گوئی شدہ امکانات سے لیے جائیں گے، اور جزوی طور پر حساب شدہ `gradients` سے۔ ہم `alpha` پیرامیٹر استعمال کریں گے تاکہ یہ طے کیا جا سکے کہ `gradients` اور انعامات کو کس حد تک مدنظر رکھا جائے - اسے تقویتی الگورتھم کی *سیکھنے کی شرح* کہا جاتا ہے۔
5. آخر میں، ہم اپنے نیٹ ورک کو حالات اور متوقع اعمال پر تربیت دیں گے، اور اس عمل کو دہرائیں گے۔


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

اب آئیے قسط کو رینڈرنگ کے ساتھ چلائیں تاکہ نتیجہ دیکھ سکیں۔


In [ ]:
_ = run_episode(render=True)

امید ہے کہ آپ دیکھ سکتے ہیں کہ اب پول کافی اچھی طرح توازن برقرار رکھ سکتا ہے!

## ایکٹر-کریٹک ماڈل

ایکٹر-کریٹک ماڈل پالیسی گریڈینٹس کی مزید ترقی ہے، جس میں ہم ایک نیورل نیٹ ورک بناتے ہیں جو پالیسی اور متوقع انعامات دونوں سیکھتا ہے۔ نیٹ ورک کے دو آؤٹ پٹس ہوں گے (یا آپ اسے دو الگ الگ نیٹ ورکس کے طور پر دیکھ سکتے ہیں):
* **ایکٹر** ہمیں اسٹیٹ پروبیبلٹی ڈسٹریبیوشن دے کر وہ عمل تجویز کرے گا جو لینا چاہیے، جیسا کہ پالیسی گریڈینٹ ماڈل میں ہوتا ہے۔
* **کریٹک** ان اعمال سے ممکنہ انعامات کا اندازہ لگائے گا۔ یہ دیے گئے اسٹیٹ پر مستقبل میں کل متوقع انعامات واپس کرے گا۔

آئیے ایسا ماڈل ڈیفائن کرتے ہیں:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

ہمیں اپنی `discounted_rewards` اور `run_episode` فنکشنز کو تھوڑا سا تبدیل کرنے کی ضرورت ہوگی:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


اب ہم مرکزی تربیتی لوپ چلائیں گے۔ ہم دستی نیٹ ورک تربیتی عمل استعمال کریں گے جس میں مناسب نقصان کے افعال کا حساب لگانا اور نیٹ ورک کے پیرامیٹرز کو اپ ڈیٹ کرنا شامل ہے۔


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## خلاصہ

ہم نے اس ڈیمو میں دو RL الگورتھمز دیکھے: سادہ پالیسی گریڈینٹ اور زیادہ پیچیدہ ایکٹر-کریٹک۔ آپ دیکھ سکتے ہیں کہ یہ الگورتھمز حالت، عمل اور انعام کے تجریدی تصورات کے ساتھ کام کرتے ہیں - اس لیے انہیں بہت مختلف ماحول میں لاگو کیا جا سکتا ہے۔

ری انفورسمنٹ لرننگ ہمیں صرف آخری انعام کو دیکھ کر مسئلہ حل کرنے کی بہترین حکمت عملی سیکھنے کی اجازت دیتا ہے۔ یہ حقیقت کہ ہمیں لیبل شدہ ڈیٹاسیٹس کی ضرورت نہیں ہوتی، ہمیں اپنے ماڈلز کو بہتر بنانے کے لیے کئی بار سیمولیشنز دہرانے کی اجازت دیتی ہے۔ تاہم، RL میں اب بھی بہت سے چیلنجز موجود ہیں، جنہیں آپ سیکھ سکتے ہیں اگر آپ اس دلچسپ AI کے شعبے پر زیادہ توجہ دینے کا فیصلہ کریں۔



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
